In [1]:
import numpy as np
import pandas as pd
import csv
from time import time

# plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train = train.drop('id', axis=1)
train = train.drop('titulo', axis=1)
train = train.drop('descripcion', axis=1)
#train = train.drop('tipodepropiedad', axis=1)
train = train.drop('direccion', axis=1)
train = train.drop('ciudad', axis=1)
#train = train.drop('provincia', axis=1)
train = train.drop('lat', axis=1)
train = train.drop('lng', axis=1)
train = train.drop('fecha', axis=1)

In [4]:
train[['antiguedad']] = train[['antiguedad']].fillna(value=81)
train[['habitaciones']] = train[['habitaciones']].fillna(value=0)
train[['garages']] = train[['garages']].fillna(value=4)
train[['banos']] = train[['banos']].fillna(value=0)
train[['metroscubiertos']] = train[['metroscubiertos']].fillna(value=0)
train[['metrostotales']] = train[['metrostotales']].fillna(value=0)
train[['idzona']] = train[['idzona']].fillna(value=0)

In [5]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [6]:
train = oneHotEncoding(train,'provincia')
train = oneHotEncoding(train,'tipodepropiedad')

In [7]:
train = train.drop('Garage', axis=1)
train = train.drop('Hospedaje', axis=1)

In [15]:
train_label = []
train_data = []

train_label = train.iloc[:, 12]
train_data = train.iloc[:, train.columns != 'precio']

In [9]:
test = pd.read_csv('test.csv')

In [10]:
test = test.drop('id', axis=1)
test = test.drop('titulo', axis=1)
test = test.drop('descripcion', axis=1)
#test = test.drop('tipodepropiedad', axis=1)
test = test.drop('direccion', axis=1)
test = test.drop('ciudad', axis=1)
#test = test.drop('provincia', axis=1)
test = test.drop('lat', axis=1)
test = test.drop('lng', axis=1)
test = test.drop('fecha', axis=1)

In [11]:
test[['antiguedad']] = test[['antiguedad']].fillna(value=81)
test[['habitaciones']] = test[['habitaciones']].fillna(value=0)
test[['garages']] = test[['garages']].fillna(value=4)
test[['banos']] = test[['banos']].fillna(value=0)
test[['metroscubiertos']] = test[['metroscubiertos']].fillna(value=0)
test[['metrostotales']] = test[['metrostotales']].fillna(value=0)
test[['idzona']] = test[['idzona']].fillna(value=0)

In [12]:
test = oneHotEncoding(test,'provincia')
test = oneHotEncoding(test,'tipodepropiedad')

In [16]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'target'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [17]:
#-----------------------------------------------------------------
# Modelo Random Forest Regressor
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [42]:
params = [ {'n_estimators': [10, 50, 100]},]

modeloRFR = RandomForestRegressor(random_state=0, n_jobs=-1)
grid = GridSearchCV(modeloRFR, params, n_jobs=-1)
grid.fit(train_data, train_label)

acc = grid.score(train_data, train_label)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

[INFO] grid search accuracy: 96.91%
[INFO] grid search best parameters: {'n_estimators': 100}


In [18]:
modeloRFR=RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=100)

In [19]:
modeloRFR.fit(train_data, train["precio"])

result = modeloRFR.predict(test)

In [20]:
write_submission("test.csv", result, "RFR_provincia_tipoDP.csv")

In [84]:
#-----------------------------------------------------------------
# Modelo GradientBoostingRegressor
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor

In [15]:
params = [ {'n_estimators': [50, 100, 150]},]

modeloGBR = GradientBoostingRegressor(random_state=0)
grid = GridSearchCV(modeloGBR, params, n_jobs=-1)
grid.fit(train_data, train_label)

acc = grid.score(train_data, train_label)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

[INFO] grid search accuracy: 66.77%
[INFO] grid search best parameters: {'n_estimators': 150}


In [16]:
modeloGBR=GradientBoostingRegressor(random_state=0, n_estimators=150)

In [17]:
modeloGBR.fit(train_data, train["precio"])

result = modeloGBR.predict(test)

In [18]:
write_submission("test.csv", result, "GBR.csv")

In [19]:
del modeloGBR

In [20]:
#-----------------------------------------------------------------
# Modelo Bagging Regressor
#-----------------------------------------------------------------
from sklearn.ensemble import BaggingRegressor

In [ ]:
params = [ {'n_estimators': [10, 50, 100]},]

modeloBG = BaggingRegressor(random_state=0)
grid = GridSearchCV(modeloBG, params, n_jobs=-1)
grid.fit(train_data, train_label)

acc = grid.score(train_data, train_label)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

In [ ]:
modeloBG=BaggingRegressor(random_state=0, n_jobs=-1, n_estimators=150)

In [ ]:
modeloBG.fit(train_data, train["precio"])

result = modeloBG.predict(test)

In [ ]:
write_submission("test.csv", result, "BG.csv")

In [ ]:
del modeloBG

In [11]:
#-----------------------------------------------------------------
# Modelo Multi-layer Perceptron regressor
#-----------------------------------------------------------------
from sklearn.neural_network import MLPRegressor

In [ ]:
params = [
    {
        'activation' : ('logistic', 'tanh', 'relu',),
        'solver' : ('lbfgs','adam','sgd',),
        'learning_rate' : ('constant', 'invscaling', 'adaptive',),
    },
]

modeloMLPR = MLPRegressor(random_state=0)
grid = GridSearchCV(modeloMLPR, params, n_jobs=-1)
grid.fit(train_data, train_label)

acc = grid.score(train_data, train_label)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

In [ ]:
modeloMLPR=MLPRegressor(random_state=0, n_estimators=150)

In [ ]:
modeloMLPR.fit(train_data, train["precio"])

result = modeloMLPR.predict(test)

In [ ]:
write_submission("test.csv", result, "MLPR.csv")